In [1]:
import pandas as pd
from time import sleep
import csv
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler

## in this script we will predict the quote labels of the whole dataset and save it as csv file

In [2]:
#load model
infile = open('BestModel','rb')
model = pickle.load(infile, encoding='bytes')

/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
#load feature object 
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)

infile2 = open('searchResults','rb')
results = pickle.load(infile2, encoding='bytes')

In [4]:
#get the first 200 labels: those are the one in the mannual annotate sample
def append_features(ob):
    proto_matrix = []
    for item in ob:
        col2 = np.append(ob[item].scores, ob[item].cos)
        #print(col2)
        proto_matrix.append(col2)
      
    return proto_matrix
 
            
proto_matrix = append_features(results)
FeatureMatrix = np.matrix(proto_matrix)

In [5]:
FeatureMatrix

matrix([[1.        , 1.        , 5.        , ..., 6.        , 0.        ,
         0.90142585],
        [5.        , 2.        , 3.        , ..., 3.        , 0.        ,
         0.97699932],
        [5.        , 7.        , 3.        , ..., 1.        , 0.        ,
         0.96864645],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         0.89188675],
        [9.        , 5.        , 9.        , ..., 0.        , 0.        ,
         0.94524863],
        [1.        , 1.        , 1.        , ..., 1.        , 0.        ,
         0.8824201 ]])

In [6]:
scaler = StandardScaler()
scaled_matrix = scaler.transform(FeatureMatrix)
scaled_matrix

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [15]:
y = model.predict(scaled_matrix)
len(y)

4520

In [24]:
def Save_Results(objects,pre_results,fname,fname2):
    with open(fname , 'w') as f:
        writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
        writer.writerow(["hash"] + ["text"] + ["score"] + ["cosineSim"])
        
        for item in objects:
            writer.writerow([objects[item].quoteID] + [objects[item].quoteText] + [objects[item].scores] + [objects[item].cos])  
    f.close()
    
    with open(fname2 , 'w') as f:
        writer = csv.writer(f, delimiter = ',',quoting=csv.QUOTE_MINIMAL)  
        writer.writerow(["result"])
        
        for item in pre_results:
            writer.writerow([item])  
    f.close()
    
Save_Results(results, y, 'text_all.csv', 'results_all.csv')